In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [5]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [6]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [7]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [8]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [10]:
# prepare embedding matrix  
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv2D, GlobalMaxPooling2D, Dense, Dropout, LSTM, Bidirectional
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
#prepare Embedding layer
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [14]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')                            ## Input layer
embedded_sequences = embedding_layer(sequence_input)                                           ## Embeddings layer
X = Bidirectional(LSTM(200, return_sequences=False), merge_mode='concat')(embedded_sequences)  ## BiLSTM layer
X = Dropout(0.5)(X)                                                                            ## Dropout layer  
X = Dense(30, activation='relu')(X)                                                            ## Fully Connected layer
X = Dropout(0.5)(X)                                                                            ## Dropout layer
preds = Dense(2, activation='softmax')(X)                                                      ## Softmax layer
model = Model(sequence_input, preds)

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9
                                   , epsilon=None, decay=0.3)



In [16]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Instructions for updating:
Use tf.cast instead.
Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.4650 - acc: 0.7796 - val_loss: 0.3808 - val_acc: 0.8182
Epoch 2/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.3831 - acc: 0.8340 - val_loss: 0.3571 - val_acc: 0.8412
Epoch 3/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3726 - acc: 0.8428 - val_loss: 0.3446 - val_acc: 0.8432
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3568 - acc: 0.8588 - val_loss: 0.3393 - val_acc: 0.8462
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3422 - acc: 0.8620 - val_loss: 0.3316 - val_acc: 0.8482
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3373 - acc: 0.8595 - val_loss: 0.3284 - val_acc: 0.8551
Epoch 7/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3372 - acc: 0.8

In [0]:
from keras import backend as K

def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


      
def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
      

def f1(y_true, y_pred):
    
    Precision = precision(y_true, y_pred)
    Recall = recall(y_true, y_pred)
    f1 = 2*((Precision*Recall)/(Precision+Recall+K.epsilon()))
    
    return f1



In [18]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2810 - precision: 0.8880 - val_loss: 0.2864 - val_precision: 0.8751
Epoch 2/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2848 - precision: 0.8898 - val_loss: 0.2859 - val_precision: 0.8741
Epoch 3/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2927 - precision: 0.8845 - val_loss: 0.2857 - val_precision: 0.8761
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2867 - precision: 0.8858 - val_loss: 0.2857 - val_precision: 0.8781
Epoch 5/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2885 - precision: 0.8898 - val_loss: 0.2852 - val_precision: 0.8771
Epoch 6/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2866 - precision: 0.8868 - val_loss: 0.2852 - val_precision: 0.8771
Epoch 7/50
4001/4001 [==============================] - 14s 4ms/step -

In [19]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = rmsprop,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2838 - recall: 0.8903 - val_loss: 0.2770 - val_recall: 0.8781
Epoch 2/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2727 - recall: 0.8933 - val_loss: 0.2770 - val_recall: 0.8781
Epoch 3/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2750 - recall: 0.8903 - val_loss: 0.2766 - val_recall: 0.8781
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2746 - recall: 0.8903 - val_loss: 0.2765 - val_recall: 0.8781
Epoch 5/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2776 - recall: 0.8923 - val_loss: 0.2762 - val_recall: 0.8781
Epoch 6/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2747 - recall: 0.8935 - val_loss: 0.2762 - val_recall: 0.8781
Epoch 7/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2772 - recall: 0.8910 - val

In [20]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = rmsprop, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2807 - f1: 0.8928 - val_loss: 0.2720 - val_f1: 0.8771
Epoch 2/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2690 - f1: 0.8973 - val_loss: 0.2719 - val_f1: 0.8781
Epoch 3/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2669 - f1: 0.8958 - val_loss: 0.2719 - val_f1: 0.8781
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2713 - f1: 0.8978 - val_loss: 0.2717 - val_f1: 0.8781
Epoch 5/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2722 - f1: 0.8953 - val_loss: 0.2715 - val_f1: 0.8781
Epoch 6/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2722 - f1: 0.8943 - val_loss: 0.2713 - val_f1: 0.8771
Epoch 7/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2715 - f1: 0.8948 - val_loss: 0.2714 - val_f1: 0.8771
Epoch 8/50
4001/4001 

In [0]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.25, amsgrad=False)

In [22]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2984 - acc: 0.8810 - val_loss: 0.2722 - val_acc: 0.8781
Epoch 2/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2524 - acc: 0.8993 - val_loss: 0.2663 - val_acc: 0.8791
Epoch 3/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2409 - acc: 0.9038 - val_loss: 0.2570 - val_acc: 0.8811
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2345 - acc: 0.9073 - val_loss: 0.2520 - val_acc: 0.8881
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2324 - acc: 0.9083 - val_loss: 0.2504 - val_acc: 0.8861
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2309 - acc: 0.9108 - val_loss: 0.2498 - val_acc: 0.8891
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2344 - acc: 0.9098 - val_loss: 0.2498 - val_acc: 0.8871
Epoch 8

In [23]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.2150 - precision: 0.9195 - val_loss: 0.2401 - val_precision: 0.8931
Epoch 2/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2090 - precision: 0.9190 - val_loss: 0.2391 - val_precision: 0.8931
Epoch 3/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2115 - precision: 0.9208 - val_loss: 0.2390 - val_precision: 0.8931
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2079 - precision: 0.9170 - val_loss: 0.2392 - val_precision: 0.8921
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2117 - precision: 0.9223 - val_loss: 0.2392 - val_precision: 0.8941
Epoch 6/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2147 - precision: 0.9205 - val_loss: 0.2390 - val_precision: 0.8931
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step -

In [24]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 17s 4ms/step - loss: 0.2072 - recall: 0.9213 - val_loss: 0.2373 - val_recall: 0.8951
Epoch 2/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2057 - recall: 0.9213 - val_loss: 0.2372 - val_recall: 0.8941
Epoch 3/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2031 - recall: 0.9255 - val_loss: 0.2379 - val_recall: 0.8951
Epoch 4/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2091 - recall: 0.9213 - val_loss: 0.2374 - val_recall: 0.8951
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2020 - recall: 0.9253 - val_loss: 0.2374 - val_recall: 0.8951
Epoch 6/50
4001/4001 [==============================] - 18s 4ms/step - loss: 0.2014 - recall: 0.9210 - val_loss: 0.2377 - val_recall: 0.8951
Epoch 7/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2026 - recall: 0.9223 - val

In [25]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = adam, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 17s 4ms/step - loss: 0.2001 - f1: 0.9243 - val_loss: 0.2377 - val_f1: 0.8971
Epoch 2/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2001 - f1: 0.9225 - val_loss: 0.2375 - val_f1: 0.8981
Epoch 3/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2028 - f1: 0.9240 - val_loss: 0.2375 - val_f1: 0.8971
Epoch 4/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.1980 - f1: 0.9238 - val_loss: 0.2375 - val_f1: 0.8981
Epoch 5/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.1982 - f1: 0.9225 - val_loss: 0.2373 - val_f1: 0.8981
Epoch 6/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.1993 - f1: 0.9250 - val_loss: 0.2373 - val_f1: 0.8981
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.1953 - f1: 0.9268 - val_loss: 0.2374 - val_f1: 0.8981
Epoch 8/50
4001/4001 

In [0]:
model.save_weights('/content/drive/My Drive/Sarcasm Detection/Plots/BiLSTM_optimized_Weights.h5')